In [6]:
import numpy as np
import cv2
from mss import mss
from PIL import Image

import torch
import torchvision

import pyautogui
import time
import win32api, win32con
from utils.torch_utils import select_device
from utils.general import non_max_suppression
from utils.augmentations import letterbox

from models.common import DetectMultiBackend
DEVICE = 'cuda'
LOAD_MODEL_FILE = "Halo640.pt"
image_size = 640


In [7]:
CHECKPOINT_NAME = LOAD_MODEL_FILE
checkpoint = torch.load(CHECKPOINT_NAME)

device = select_device('')

model = DetectMultiBackend(CHECKPOINT_NAME, device=device, dnn=False, data='Halodata.yaml', fp16=False)
stride, names, pt = model.stride, model.names, model.pt

model.eval()


YOLOv5  2022-4-17 torch 1.11.0 CUDA:0 (NVIDIA GeForce RTX 3070 Laptop GPU, 8192MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients


DetectMultiBackend(
  (model): Model(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(
        

In [8]:
# function to convert a torchtensor back to PIL image
def torch_to_pil(img):
    return torchvision.transforms.ToPILImage()(img).convert('RGB')


In [10]:
show = True
dbg = True

#centering is how much of the screen to account for, from the center
centering = 1 # how much of the screen to grab

threshold = .5
sct = mss()

movement_scale = 8

w, h = pyautogui.size()

x_center = w/2
y_center = h/2

top = int(y_center - (y_center * centering))
left = int(x_center - (x_center * centering))
bottom = int(y_center + (y_center * centering))
right = int(x_center + (x_center * centering))

width = int(w * centering)
height = int(h * centering)
monitor = {'top': top, 'left': left, 'width': width, 'height': height}

model.to(DEVICE)

while True:
    if dbg:
        start = time.time()

    orig_img = sct.grab(monitor)
    img = np.array(Image.frombytes('RGB', (orig_img.size.width, orig_img.size.height), orig_img.rgb))

    # Padded resize
    img = letterbox(img, (image_size, image_size), stride=stride, auto=pt)[0]



    # Convert
    img = img.transpose((2, 0, 1))[::-1]  # HWC to CHW, BGR to RGB
    img = np.ascontiguousarray(img)
    img = torch.from_numpy(img).to(device).float()

    img /= 255  # 0 - 255 to 0.0 - 1.0

    with torch.no_grad():
        prediction = model(img.unsqueeze(0))

    pred = non_max_suppression(prediction)[0]

    img = np.array(torch_to_pil(img))

    if dbg:
        framerate = 1/(time.time() - start)


    #add framerate to the cv2 image
    cv2.putText(img, str(framerate), (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                   1, (255, 0, 0), 2, cv2.LINE_AA)
    if len(pred) > 0:
        for box in pred:
            if box[4] > threshold:
                box_dim = (box[3] - box[1])  #also used to scale inputs later on
                x = (box[2] + box[0]) / 2
                y = ((box[3] + box[1]) / 2) - ((box[3] - box[1]) / 3)
                cv2.circle(img,
                    (int(x), int(y)),
                    radius=10,
                    color=(255, 0, 0),
                    thickness=5
                )

                #get the coordinates in the full size space
                x = x / img.shape[1] * orig_img.size[0]

                y = y / img.shape[0] * orig_img.size[1]

                win32api.mouse_event(win32con.MOUSEEVENTF_MOVE, int(movement_scale*box_dim/orig_img.size.height*(x.item()-x_center)), int(movement_scale*box_dim/orig_img.size.height*(y.item()-y_center)))

    if show:
        #img = cv2.resize(img, (orig_img.size.width, orig_img.size.height))
        cv2.imshow('test', np.array(img))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

    else:
        print(framerate)

In [ ]:

# # to view test images
#
# import glob
# # for img in glob.glob('C:/Users/blain/Documents\Git\AutoAim\model\halo_data/*.png'):
# for img in glob.glob('C:/Users/blain\Documents\Git\AutoAim\model\halo_data/new/*'):
#
#     img = Image.open(img)
#     img = img.resize(resize)
#     img_transform = transform(img).unsqueeze(0).cuda()
#     img = np.array(img)
#
#
#     boxes = utils.cellboxes_to_boxes(model(img_transform))
#     boxes = utils.non_max_suppression(boxes[0], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
#     primary, secondary = utils.pick_targets(img, boxes)
#
#     # add aim points to image
#     for aimpoint in primary:
#         print(aimpoint)
#         cv2.circle(img,
#             (int(aimpoint[0]), int(aimpoint[1])),
#             radius=10,
#             color=(255, 0, 0),
#             thickness=5
#         )
#     cv2.imshow('test', cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR))
#
#     if cv2.waitKey(0) & 0xFF == ord('q'):
#
#         break
# cv2.destroyAllWindows()